NGUYỄN TRỌNG HUY - MSSV 51900507


In [68]:
from flask import Flask, request, jsonify
import joblib

In [69]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [70]:
# Tạo dữ liệu mẫu cho tệp data.csv
np.random.seed(42)  
n_samples = 1000
x1 = np.random.rand(n_samples) * 10  
x2 = np.random.rand(n_samples) * 5   
noise = np.random.normal(0, 1, n_samples)  
coefficients = [2, 3]
y = coefficients[0] * x1 + coefficients[1] * x2 + noise
data = pd.DataFrame({'x1': x1, 'x2': x2, 'y': y})

data.to_csv('data.csv', index=False)

In [71]:
# Tạo dữ liệu mẫu cho tệp data2.csv
new_data_size = 200
new_data = pd.DataFrame({
    'x1': np.random.rand(new_data_size),
    'x2': np.random.rand(new_data_size),
    'y': 2 * np.random.rand(new_data_size) + 3
})

new_data.to_csv('data2.csv', index=False)

In [72]:
data = pd.read_csv('data.csv')

In [73]:
data.head()

,x1,x2,y
0,3.745401,0.925665,9.389814
1,9.507143,2.709505,26.315920
2,7.319939,4.364729,27.507587
3,5.986585,3.661124,23.323908
4,1.560186,4.032806,16.132375


In [74]:
X_train, X_test, y_train, y_test = train_test_split(data[['x1', 'x2']], data['y'], test_size=0.25)


 Create the linear regression model

In [75]:
model = LinearRegression()

In [76]:
# Fit the model to the training data
model.fit(X_train, y_train)

LinearRegression()

In [77]:
# Make predictions on the test data
y_pred = model.predict(X_test)

In [78]:
# Evaluate the model
print('Mean squared error:', np.mean((y_test - y_pred) ** 2))
print('Root mean squared error:', np.sqrt(np.mean((y_test - y_pred) ** 2)))


Mean squared error: 0.8784254428802071
Root mean squared error: 0.937243534456337


In [79]:
# Save the model using joblib
joblib.dump(model, 'model.pkl')

['model.pkl']

In [80]:
# Continual learning
# Load the saved model
loaded_model = joblib.load('model.pkl')

In [81]:
# Train the model on new data
data2 = pd.read_csv('data2.csv')
loaded_model.fit(data2[['x1', 'x2']], data2['y'])

LinearRegression()

In [82]:
# Make predictions on new data
new_y_pred = loaded_model.predict(data2[['x1', 'x2']])


In [83]:
# Evaluate the loaded model on new data
print('Mean squared error (continual learning):', np.mean((data2['y'] - new_y_pred) ** 2))
print('Root mean squared error (continual learning):', np.sqrt(np.mean((data2['y'] - new_y_pred) ** 2)))

Mean squared error (continual learning): 0.3291558210985028
Root mean squared error (continual learning): 0.5737210307270448


In [84]:
# Test production
# Deploy the model to a web service
app = Flask(__name__)

In [85]:
@app.route('/predict', methods=['POST'])
def predict():
    # Get the input data
    data = request.get_json()

    # Make predictions using the loaded model
    predictions = loaded_model.predict([[data['x1'], data['x2']]])

    # Return the predictions as JSON
    return jsonify({'prediction': predictions[0]})

if __name__ == '__main__':
    app.run(debug=True)